In [1]:
import csv
import datetime


### 0. url-subject map 생성.
### 1. 먼저 class-central review platform 돌면서 pattern 추출,
### 2. 다음으로 coursetalk review platform.


In [2]:
def extract_pattern(nodes, reviewer_id):
    if len(nodes) > 0:
        print(nodes)
    tempDict = {}
    # nodes 사이즈가 0인 경우 return
    if len(nodes) == 0:
        return tempDict
    
    # nodes 사이즈가 1인 경우, subjectMap에 과목이 있을 경우 transcriptList에 추가.
    if len(nodes) == 1:
        tempDict = {
            'id': reviewer_id,
            '1': [nodes[0]['course']]
        }
        return tempDict
            
        
    # nodes 사이즈가 2 이상인 경우, transcipt list setting.
    else:
        iterator = 0;
        # item sets number init as 1
        set_num = 1 
        # temp transcript dictionary init
        tempDict = {
            'id': reviewer_id,
            '1': [nodes[0]['course']] # 첫번재 인덱스의 course를 넣고 시작.
        }
        for i, node in enumerate(nodes):
#             print("iterator ", iterator, ", i=", i, ", length- ", len(nodes))
            if iterator == len(nodes):
#                 print("iter:", iterator, "==(length",len(nodes), ")break!")
                break
                
            if i < iterator: # 중복으로 검사하지 않기위해 i를 현재 iterator까지 증가시킴. 
                continue

            for j, node_for_comparision in enumerate(nodes):
                
                # j가 i보다 작을 경우 continue
                if not j > i:
                    continue
                
                # review date가 같을 경우, set-i에 이미 존재하는지 확인하고 넣기.                                   
                if node['date'] == node_for_comparision['date']:
#                     print(node['course'] , " - " , node_for_comparision['course'], " is same, append current set-", str(set_num))
                    tempDict[str(set_num)].append(node_for_comparision['course'])
                    iterator += 1
                    
                # date가 차이가 날 경우, 현재 anchor의 date가 더 작을 경우만 고려함. 이미 date순으로 정렬되어 있기 때문.
                elif node['date'] < node_for_comparision['date']:
#                     print(node['course'] , " - " , node_for_comparision['course'], " is diff, append current set-",str(set_num+1))
                    set_num += 1
                    tempDict[str(set_num)] = [node_for_comparision['course']]
                    iterator += 1
                    break
                        
        return tempDict
            

In [3]:
def transcript_to_subject_mapping(transList):
    
    for index, trans in enumerate(transList):
#         print(trans)
        set_len = len(trans.keys())
        for i in range(1, set_len):
            
            for j, item in enumerate(trans[str(i)]):
#                 print(item)
                if item not in subjectMap.keys():
#                     print("remove the item-", item)
                    transList[index][str(i)].pop(j)
                else:
                    transList[index][str(i)][j] = subjectMap[item]               
       
    return transList;


In [4]:
def sort_by_num(x):
    if x=="id":
        return 0
    else:
        return int(x)


In [5]:
subjectMap = {}
with open("./MOOC_pattern_extraction/url_subject_info.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        subjectMap[row['url']] = row['subject']

In [6]:
## read course Data set
courseList = []
with open("./MOOC_data/xtData/(cleand)ClassCentral_course.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        courseList.append(row)
        
print(len(courseList))

3153


In [7]:
## read review Data set
reviewList = []
with open("./MOOC_data/xtData/(Date_Adjusted)ClassCentral_review_renewed.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        reviewList.append(row)
print(len(reviewList))

20903


<h3> Extract Pattern </h3>

In [8]:
## 사용자, 사용자의 수강리스트 만들기.

reviewers =[]
for review in reviewList:
    
    
    # reviewer id가 없을 경우 continue
    if review['reviewer_url'] == "null":
        continue
        
    
    # reviewers list 를 돌면서 course 추가
    flag = False
    for index, re in enumerate(reviewers):
        
        
        if re['reviewer'] == review['reviewer_url']:
            reviewers[index]['courses'].append(dict({
                    'date': review['review_date'],
                    'course': review['url'],
                    
                
                }))
            flag = True
            
            
    
    if flag == False:
        reviewers.append(dict({
            'reviewer': review['reviewer_url'],
            'courses': [dict({
                'date': review['review_date'],
                'course': review['url'],
                
            })]
            
        }))
        

In [9]:
final_t1_trans = []
final_t2_trans = []
final_t3_trans = []
final_t4_trans = []

# course directed edge list 생성
for re in reviewers:

    t1_nodes = []
    t2_nodes = []
    t3_nodes = []
    t4_nodes = []
       
    for co in re['courses']:

               
        date = datetime.datetime.strptime(co['date'], "%Y-%m-%d") 
        
        if date.year == 2013:

            # T1: 
            if date.month > 6 and date.month <= 12:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
        
        elif date.year == 2014:
            
            if date.month > 0 and date.month < 7:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T2: 
            else:
                t2_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                

        elif date.year == 2015:
            
            if date.month > 0 and date.month < 7:
                t2_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T3: 15.7~12
            else:
                t3_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2016:
            
            if date.month > 0 and date.month < 7:
                t3_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T4
            else:
                t4_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2017:
            
            if date.month > 0 and date.month < 7:
            
                t4_nodes.append(dict({
                        'course': co['course'],
                        'date': co['date']           
                    }))
    
    # 날짜별 sorting
    t1_nodes.sort(key=lambda item:item['date'], reverse=False)
    t2_nodes.sort(key=lambda item:item['date'], reverse=False)
    t3_nodes.sort(key=lambda item:item['date'], reverse=False)
    t4_nodes.sort(key=lambda item:item['date'], reverse=False)
    
    # 같은 vertex time에 있는 노드가 두개 이상인지 체크, 2개 이상일 경우 date 비교 후 edgelist에 추가
    temp_dict1 = extract_pattern(t1_nodes, re['reviewer'])
    temp_dict2 = extract_pattern(t2_nodes, re['reviewer'])
    temp_dict3 = extract_pattern(t3_nodes, re['reviewer'])
    temp_dict4 = extract_pattern(t4_nodes, re['reviewer'])
    if temp_dict1:
        final_t1_trans.append(temp_dict1)
    if temp_dict2:
        final_t2_trans.append(temp_dict2)
    if temp_dict3:
        final_t3_trans.append(temp_dict3)
    if temp_dict4:
        final_t4_trans.append(temp_dict4)
    
       


[{'date': '2017-02-08', 'course': 'https://www.class-central.com/mooc/6344/edx-toefl-test-preparation-the-insider-s-guide'}]
[{'date': '2017-04-08', 'course': 'https://www.class-central.com/mooc/6344/edx-toefl-test-preparation-the-insider-s-guide'}, {'date': '2017-04-08', 'course': 'https://www.class-central.com/mooc/1811/coursera-personal-family-financial-planning'}, {'date': '2017-04-08', 'course': 'https://www.class-central.com/mooc/4292/coursera-front-end-javascript-frameworks-angularjs'}, {'date': '2017-04-08', 'course': 'https://www.class-central.com/mooc/1712/coursera-the-data-scientist-s-toolbox'}, {'date': '2017-04-08', 'course': 'https://www.class-central.com/mooc/1713/coursera-r-programming'}, {'date': '2017-04-08', 'course': 'https://www.class-central.com/mooc/1714/coursera-getting-and-cleaning-data'}, {'date': '2017-04-08', 'course': 'https://www.class-central.com/mooc/2161/coursera-learning-how-to-learn-powerful-mental-tools-to-help-you-master-tough-subjects'}, {'date': '

[{'date': '2015-05-19', 'course': 'https://www.class-central.com/mooc/706/coursera-competitive-strategy'}]
[{'date': '2015-06-23', 'course': 'https://www.class-central.com/mooc/706/coursera-competitive-strategy'}]
[{'date': '2015-07-31', 'course': 'https://www.class-central.com/mooc/725/coursera-introduction-to-environmental-law-and-policy'}]
[{'date': '2014-12-29', 'course': 'https://www.class-central.com/mooc/1522/coursera-marine-megafauna-an-introduction-to-marine-science-and-conservation'}, {'date': '2015-04-19', 'course': 'https://www.class-central.com/mooc/2161/coursera-learning-how-to-learn-powerful-mental-tools-to-help-you-master-tough-subjects'}, {'date': '2015-06-13', 'course': 'https://www.class-central.com/mooc/1368/coursera-bioinformatic-methods-i'}, {'date': '2015-06-13', 'course': 'https://www.class-central.com/mooc/564/coursera-introduction-to-engineering-mechanics'}, {'date': '2015-06-15', 'course': 'https://www.class-central.com/mooc/2728/coursera-initiating-and-plann

[{'date': '2015-07-27', 'course': 'https://www.class-central.com/mooc/2161/coursera-learning-how-to-learn-powerful-mental-tools-to-help-you-master-tough-subjects'}]
[{'date': '2016-05-14', 'course': 'https://www.class-central.com/mooc/2161/coursera-learning-how-to-learn-powerful-mental-tools-to-help-you-master-tough-subjects'}]
[{'date': '2015-07-27', 'course': 'https://www.class-central.com/mooc/2161/coursera-learning-how-to-learn-powerful-mental-tools-to-help-you-master-tough-subjects'}]
[{'date': '2016-09-08', 'course': 'https://www.class-central.com/mooc/2161/coursera-learning-how-to-learn-powerful-mental-tools-to-help-you-master-tough-subjects'}]
[{'date': '2017-05-08', 'course': 'https://www.class-central.com/mooc/2161/coursera-learning-how-to-learn-powerful-mental-tools-to-help-you-master-tough-subjects'}]
[{'date': '2015-07-28', 'course': 'https://www.class-central.com/mooc/2161/coursera-learning-how-to-learn-powerful-mental-tools-to-help-you-master-tough-subjects'}]
[{'date': 

In [10]:
t1_trans_final = transcript_to_subject_mapping(final_t1_trans)
t2_trans_final = transcript_to_subject_mapping(final_t2_trans)
t3_trans_final = transcript_to_subject_mapping(final_t3_trans)
t4_trans_final = transcript_to_subject_mapping(final_t4_trans)


In [11]:
# extract field names set
set1 = set()
for trans in t1_trans_final:
    for key in trans.keys():
        set1.add(key)
    
set2 = set()
for trans in t2_trans_final:
    for key in trans.keys():
        set2.add(key)
    
    
set3 = set()
for trans in t3_trans_final:
    for key in trans.keys():
        set3.add(key)
    
    
set4 = set()
for trans in t4_trans_final:
    for key in trans.keys():
        set4.add(key)


In [12]:
# write sequence list
with open('./MOOC_pattern_extraction/classCentral_pattern(sequence_by_reviewdate)_t1.csv', 'w', newline='') as write_file:
    fieldnames = list(set1)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t1_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/classCentral_pattern(sequence_by_reviewdate)_t2.csv', 'w', newline='') as write_file:
    fieldnames = list(set2)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t2_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/classCentral_pattern(sequence_by_reviewdate)_t3.csv', 'w', newline='') as write_file:
    fieldnames = list(set3)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t3_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/classCentral_pattern(sequence_by_reviewdate)_t4.csv', 'w', newline='') as write_file:
    fieldnames = list(set4)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    
    dict_writer.writeheader()
    for row in t4_trans_final:
        dict_writer.writerow(row)

### extract pattern at courseTalk

In [13]:
# data load

courseList = []
with open("./MOOC_data/xtData/(cleand_added_missing)Coursetalk_course.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row['url'] = row['url'].replace("/enroll?course_id=", "")
        row['url'] = row['url'].replace("&direct_enroll=true", "")
           
        courseList.append(row)
        
print(len(courseList))

## read review Data set
reviewList = []
with open("./MOOC_data/xtData/CourseTalk_review_renewed.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row['review_value'] = int(row['review_value'])/2 # 0~10 scale이므로 2로 나누어 0~5 scale로 변환.
        reviewList.append(row)
print(len(reviewList))

3331
23604


In [14]:
## 사용자, 사용자의 수강리스트 만들기.

reviewers =[]
for review in reviewList:
    
    
    # reviewer id가 없을 경우 continue
    if review['reviewer_url'] == "null":
        continue
        
    
    # reviewers list 를 돌면서 course 추가
    flag = False
    for index, re in enumerate(reviewers):
                
        if re['reviewer'] == review['reviewer_url']:
            reviewers[index]['courses'].append(dict({
                    'date': review['review_date'],
                    'course': review['course_id']
                }))
            flag = True
            
            
    
    if flag == False:
        reviewers.append(dict({
            'reviewer': review['reviewer_url'],
            'courses': [dict({
                'date': review['review_date'],
                'course': review['course_id']
            })]
            
        }))
        

In [15]:
final_t1_trans = []
final_t2_trans = []
final_t3_trans = []
final_t4_trans = []

# course directed edge list 생성
for re in reviewers:

    t1_nodes = []
    t2_nodes = []
    t3_nodes = []
    t4_nodes = []
       
    for co in re['courses']:

               
        date = datetime.datetime.strptime(co['date'], "%Y-%m-%d") 
        
        if date.year == 2013:

            # T1: 
            if date.month > 6 and date.month <= 12:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
        
        elif date.year == 2014:
            
            if date.month > 0 and date.month < 7:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T2: 
            else:
                t2_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                

        elif date.year == 2015:
            
            if date.month > 0 and date.month < 7:
                t2_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T3: 15.7~12
            else:
                t3_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2016:
            
            if date.month > 0 and date.month < 7:
                t3_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T4
            else:
                t4_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2017:
            
            if date.month > 0 and date.month < 7:
            
                t4_nodes.append(dict({
                        'course': co['course'],
                        'date': co['date']           
                    }))
    
    # 날짜별 sorting
    t1_nodes.sort(key=lambda item:item['date'], reverse=False)
    t2_nodes.sort(key=lambda item:item['date'], reverse=False)
    t3_nodes.sort(key=lambda item:item['date'], reverse=False)
    t4_nodes.sort(key=lambda item:item['date'], reverse=False)
    
    # 같은 vertex time에 있는 노드가 두개 이상인지 체크, 2개 이상일 경우 date 비교 후 edgelist에 추가
    temp_dict1 = extract_pattern(t1_nodes, re['reviewer'])
    temp_dict2 = extract_pattern(t2_nodes, re['reviewer'])
    temp_dict3 = extract_pattern(t3_nodes, re['reviewer'])
    temp_dict4 = extract_pattern(t4_nodes, re['reviewer'])
    if temp_dict1:
        final_t1_trans.append(temp_dict1)
    if temp_dict2:
        final_t2_trans.append(temp_dict2)
    if temp_dict3:
        final_t3_trans.append(temp_dict3)
    if temp_dict4:
        final_t4_trans.append(temp_dict4)
    
       


[{'date': '2016-01-05', 'course': '47046'}]
[{'date': '2013-12-27', 'course': '81876'}]
[{'date': '2014-12-09', 'course': '16'}, {'date': '2015-01-19', 'course': '68064'}, {'date': '2015-05-31', 'course': '112'}]
[{'date': '2015-07-20', 'course': '188'}, {'date': '2015-11-05', 'course': '83277'}, {'date': '2015-12-14', 'course': '45502'}, {'date': '2016-01-10', 'course': '26582'}, {'date': '2016-01-30', 'course': '70223'}, {'date': '2016-03-16', 'course': '44776'}, {'date': '2016-03-31', 'course': '44777'}, {'date': '2016-04-08', 'course': '44778'}, {'date': '2016-04-10', 'course': '45431'}, {'date': '2016-04-10', 'course': '45401'}, {'date': '2016-04-13', 'course': '50753'}, {'date': '2016-04-13', 'course': '50532'}, {'date': '2016-05-20', 'course': '80158'}, {'date': '2016-06-23', 'course': '89756'}]
[{'date': '2016-07-20', 'course': '69742'}, {'date': '2016-08-31', 'course': '45322'}, {'date': '2016-09-02', 'course': '36266'}, {'date': '2016-09-02', 'course': '69292'}, {'date': '201

[{'date': '2015-07-17', 'course': '36320'}]
[{'date': '2015-07-17', 'course': '36320'}]
[{'date': '2015-07-17', 'course': '36320'}]
[{'date': '2015-07-16', 'course': '36320'}]
[{'date': '2015-07-15', 'course': '36320'}]
[{'date': '2015-03-20', 'course': '36320'}]
[{'date': '2015-03-20', 'course': '36320'}]
[{'date': '2015-03-20', 'course': '36320'}]
[{'date': '2015-03-06', 'course': '36320'}]
[{'date': '2015-03-06', 'course': '36320'}]
[{'date': '2014-05-21', 'course': '30'}, {'date': '2014-06-03', 'course': '31'}]
[{'date': '2014-12-01', 'course': '25295'}]
[{'date': '2015-12-25', 'course': '50878'}, {'date': '2016-03-24', 'course': '50635'}]
[{'date': '2017-05-09', 'course': '70680'}, {'date': '2017-05-09', 'course': '50741'}]
[{'date': '2016-01-10', 'course': '51366'}]
[{'date': '2017-03-08', 'course': '78562'}]
[{'date': '2017-06-29', 'course': '70219'}]
[{'date': '2017-06-25', 'course': '70219'}]
[{'date': '2017-06-25', 'course': '70219'}]
[{'date': '2017-06-24', 'course': '70219'

[{'date': '2015-01-05', 'course': '39853'}]
[{'date': '2015-01-07', 'course': '34333'}, {'date': '2015-01-07', 'course': '27928'}]
[{'date': '2017-03-20', 'course': '75686'}]
[{'date': '2017-06-07', 'course': '67420'}]
[{'date': '2016-09-07', 'course': '50650'}]
[{'date': '2016-08-23', 'course': '50612'}]
[{'date': '2013-12-30', 'course': '12652'}]
[{'date': '2016-01-25', 'course': '28802'}]


In [16]:
t1_trans_final_courseTalk = transcript_to_subject_mapping(final_t1_trans)
t2_trans_final_courseTalk = transcript_to_subject_mapping(final_t2_trans)
t3_trans_final_courseTalk = transcript_to_subject_mapping(final_t3_trans)
t4_trans_final_courseTalk = transcript_to_subject_mapping(final_t4_trans)


In [17]:
# extract field names set
set1_courserTalk = set()
for trans in t1_trans_final_courseTalk:
    for key in trans.keys():
        set1_courserTalk.add(key)
    
set2_courserTalk = set()
for trans in t2_trans_final_courseTalk:
    for key in trans.keys():
        set2_courserTalk.add(key)
    
    
set3_courserTalk = set()
for trans in t3_trans_final_courseTalk:
    for key in trans.keys():
        set3_courserTalk.add(key)
    
    
set4_courserTalk = set()
for trans in t4_trans_final_courseTalk:
    for key in trans.keys():
        set4_courserTalk.add(key)


In [18]:
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_pattern(sequence_by_reviewdate)_t1.csv', 'w', newline='') as write_file:
    fieldnames = list(set1_courserTalk)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t1_trans_final_courseTalk:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_pattern(sequence_by_reviewdate)_t2.csv', 'w', newline='') as write_file:
    fieldnames = list(set2_courserTalk)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t2_trans_final_courseTalk:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_pattern(sequence_by_reviewdate)_t3.csv', 'w', newline='') as write_file:
    fieldnames = list(set3_courserTalk)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t3_trans_final_courseTalk:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_pattern(sequence_by_reviewdate)_t4.csv', 'w', newline='') as write_file:
    fieldnames = list(set4_courserTalk)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    
    dict_writer.writeheader()
    for row in t4_trans_final_courseTalk:
        dict_writer.writerow(row)

### Integrate transaction data

In [19]:
t1_trans_final.extend(t1_trans_final_courseTalk)
t2_trans_final.extend(t2_trans_final_courseTalk)
t3_trans_final.extend(t3_trans_final_courseTalk)
t4_trans_final.extend(t4_trans_final_courseTalk)


In [20]:
# extract field names set
set1_integrated = set()
for trans in t1_trans_final:
    for key in trans.keys():
        set1_integrated.add(key)
    
set2_integrated = set()
for trans in t2_trans_final:
    for key in trans.keys():
        set2_integrated.add(key)
    
    
set3_integrated = set()
for trans in t3_trans_final:
    for key in trans.keys():
        set3_integrated.add(key)
    
    
set4_integrated = set()
for trans in t4_trans_final:
    for key in trans.keys():
        set4_integrated.add(key)


In [21]:
# write sequence list
with open('./MOOC_pattern_extraction/integrated_pattern(sequence_by_reviewdate)_t1.csv', 'w', newline='') as write_file:
    fieldnames = list(set1_integrated)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t1_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/integrated_pattern(sequence_by_reviewdate)_t2.csv', 'w', newline='') as write_file:
    fieldnames = list(set2_integrated)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t2_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/integrated_pattern(sequence_by_reviewdate)_t3.csv', 'w', newline='') as write_file:
    fieldnames = list(set3_integrated)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t3_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/integrated_pattern(sequence_by_reviewdate)_t4.csv', 'w', newline='') as write_file:
    fieldnames = list(set4_integrated)
    fieldnames.sort(key = sort_by_num)
    dict_writer = csv.DictWriter(write_file, fieldnames)
    
    dict_writer.writeheader()
    for row in t4_trans_final:
        dict_writer.writerow(row)